# PM2.5 Prediction - Data Preprocessing

Notebook này thực hiện tiền xử lý dữ liệu với PySpark:
1. Kết nối Spark cluster
2. Đọc và khám phá dữ liệu
3. Tổng quan về dataset
4. **Làm sạch dữ liệu** (Outlier Removal → Missing Value Imputation)
5. Feature engineering
6. Data summary & statistics
7. Lưu dữ liệu đã xử lý

In [ ]:
import os
import sys

# Detect environment (Kaggle vs Colab vs Local)
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IN_COLAB = 'google.colab' in sys.modules

if IN_KAGGLE:
    print("🏆 Running on Kaggle")
    
    # Kaggle has Java pre-installed, just set JAVA_HOME
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ.get("PATH", "")
    
    # Install PySpark
    print("📦 Installing PySpark...")
    !pip install -q pyspark
    print("✓ PySpark installed")
    print(f"✓ Java: {os.environ['JAVA_HOME']}")
    
elif IN_COLAB:
    print("🌐 Running on Google Colab")
    
    # Install Java 11 (required for PySpark)
    print("📦 Installing Java 11...")
    !apt-get install -y openjdk-11-jdk-headless -qq > /dev/null
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ.get("PATH", "")
    print(f"✓ Java installed: {os.environ['JAVA_HOME']}")
    
    # Install PySpark
    print("📦 Installing PySpark...")
    !pip install -q pyspark
    print("✓ PySpark installed")
    
    # Mount Google Drive (optional - if data is in Drive)
    # from google.colab import drive
    # drive.mount('/content/drive')
    
else:
    print("💻 Running on Local Machine")
    
    # Set Java 21 for PySpark (local only)
    os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-21'
    os.environ['PATH'] = os.environ['JAVA_HOME'] + r'\bin;' + os.environ.get('PATH', '')
    print(f"✓ Using Java: {os.environ['JAVA_HOME']}")

# Common imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')

print("✓ All imports successful!")

💻 Running on Local Machine
✓ Using Java: C:\Program Files\Java\jdk-21
✓ All imports successful!


## 1. Kết nối Spark Cluster

In [ ]:
# Tạo Spark Session với cấu hình tùy theo môi trường
if IN_KAGGLE:
    # Kaggle configuration - Balanced (4 cores, 16GB RAM)
    spark = SparkSession.builder \
        .appName("PM25-Preprocessing") \
        .master("local[4]") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "6g") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.shuffle.partitions", "8") \
        .config("spark.default.parallelism", "8") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .getOrCreate()
    
    print("🏆 Spark running on Kaggle (4 cores, 12GB total)")
    
elif IN_COLAB:
    # Colab configuration - Lighter settings (2 cores, 12GB RAM)
    spark = SparkSession.builder \
        .appName("PM25-Preprocessing") \
        .master("local[2]") \
        .config("spark.driver.memory", "2g") \
        .config("spark.executor.memory", "2g") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .getOrCreate()
    
    print("🌐 Spark running on Colab (2 cores, 4GB total)")
    
else:
    # Local configuration - OPTIMIZED for 8-core AMD Ryzen + 15.7GB RAM
    spark = SparkSession.builder \
        .appName("PM25-Preprocessing") \
        .master("local[8]") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "4g") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.shuffle.partitions", "16") \
        .config("spark.default.parallelism", "16") \
        .config("spark.python.worker.timeout", "600") \
        .config("spark.executor.heartbeatInterval", "60s") \
        .config("spark.network.timeout", "600s") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true") \
        .getOrCreate()
    
    print("💻 Spark running on Local (8 cores, 12GB total - OPTIMIZED)")

print(f"✓ Spark version: {spark.version}")
print(f"✓ Spark mode: {spark.sparkContext.master}")
print(f"✓ Application ID: {spark.sparkContext.applicationId}")
print(f"✓ Cores: {spark.sparkContext.defaultParallelism}")
print(f"✓ Parallelism: {spark.conf.get('spark.default.parallelism', 'default')}")

💻 Spark running on Local (4GB memory)
✓ Spark version: 4.0.1
✓ Spark mode: local[*]
✓ Application ID: local-1762619080056
✓ Cores: 12


## 2. Định nghĩa Schema và Scan Files

In [3]:
# Schema cho dữ liệu OpenAQ
openaq_schema = StructType([
    StructField("location_id", StringType(), True),
    StructField("sensors_id", StringType(), True),
    StructField("location", StringType(), True),
    StructField("datetime", TimestampType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True),
    StructField("parameter", StringType(), True),
    StructField("units", StringType(), True),
    StructField("value", DoubleType(), True)
])

# Schema cho dữ liệu Weather
weather_schema = StructType([
    StructField("time", TimestampType(), True),
    StructField("temperature_2m", DoubleType(), True),
    StructField("relative_humidity_2m", DoubleType(), True),
    StructField("wind_speed_10m", DoubleType(), True),
    StructField("wind_direction_10m", DoubleType(), True),
    StructField("surface_pressure", DoubleType(), True),
    StructField("precipitation", DoubleType(), True)
])

print("✓ Schemas defined")

✓ Schemas defined


### 2.1 Scan và Map Files theo Location

In [ ]:
import glob
import re
from pathlib import Path

# ========================================
# KAGGLE: Sử dụng đường dẫn Kaggle dataset
# ========================================
if IN_KAGGLE:
    # 🏆 Kaggle paths
    # Format: /kaggle/input/{dataset-name}/
    raw_data_path = Path("/kaggle/input/pm25-hongkong-raw")  # ← Thay tên dataset của bạn
    print(f"🏆 Using Kaggle dataset: {raw_data_path}")
    
elif IN_COLAB:
    # 🌐 Colab: Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    raw_data_path = Path("/content/drive/MyDrive/pm25-data/raw")  # ← Thay đường dẫn Drive của bạn
    print(f"🌐 Using Google Drive: {raw_data_path}")
    
else:
    # 💻 Local path (giữ nguyên)
    raw_data_path = Path("../data/raw")
    print(f"💻 Using local path: {raw_data_path}")

# Tìm tất cả các file pollutant
pollutant_files = list(raw_data_path.glob("pollutant_location_*.csv"))

print(f"📁 Found {len(pollutant_files)} pollutant files:")

# Tạo mapping giữa pollutant và weather files
location_mapping = {}

for pollutant_file in pollutant_files:
    # Extract location_id từ tên file: pollutant_location_7727.csv → 7727
    match = re.search(r'pollutant_location_(\d+)\.csv', pollutant_file.name)
    
    if match:
        location_id = match.group(1)
        weather_file = raw_data_path / f"weather_location_{location_id}.csv"
        
        # Kiểm tra file weather tương ứng có tồn tại không
        if weather_file.exists():
            location_mapping[location_id] = {
                'pollutant': str(pollutant_file),
                'weather': str(weather_file)
            }
            print(f"  ✓ Location {location_id}: {pollutant_file.name} + {weather_file.name}")
        else:
            print(f"  ⚠️  Location {location_id}: Missing weather file!")

print(f"\n✅ Total locations to process: {len(location_mapping)}")

📁 Found 14 pollutant files:
  ✓ Location 233335: pollutant_location_233335.csv + weather_location_233335.csv
  ✓ Location 233336: pollutant_location_233336.csv + weather_location_233336.csv
  ✓ Location 7727: pollutant_location_7727.csv + weather_location_7727.csv
  ✓ Location 7728: pollutant_location_7728.csv + weather_location_7728.csv
  ✓ Location 7730: pollutant_location_7730.csv + weather_location_7730.csv
  ✓ Location 7732: pollutant_location_7732.csv + weather_location_7732.csv
  ✓ Location 7733: pollutant_location_7733.csv + weather_location_7733.csv
  ✓ Location 7734: pollutant_location_7734.csv + weather_location_7734.csv
  ✓ Location 7735: pollutant_location_7735.csv + weather_location_7735.csv
  ✓ Location 7736: pollutant_location_7736.csv + weather_location_7736.csv
  ✓ Location 7737: pollutant_location_7737.csv + weather_location_7737.csv
  ✓ Location 7739: pollutant_location_7739.csv + weather_location_7739.csv
  ✓ Location 7740: pollutant_location_7740.csv + weather_loc

### 2.2 Xử lý từng Location

In [5]:
# List để chứa dataframes của từng location
all_locations_data = []

for location_id, files in location_mapping.items():
    print(f"\n🔄 Processing Location {location_id}...")
    
    # Đọc pollutant data
    df_air = spark.read.csv(
        files['pollutant'],
        header=True,
        schema=openaq_schema
    )
    
    # 🔍 LỌC CHỈ LẤY CÁC CHỈ SỐ QUAN TÂM: PM2.5, PM10, SO2, NO2
    df_air = df_air.filter(
        F.col("parameter").isin(["pm25", "pm10", "so2", "no2"])
    )
    
    # Đọc weather data
    df_weather = spark.read.csv(
        files['weather'],
        header=True,
        schema=weather_schema
    )
    
    print(f"  📊 Air quality (PM2.5, PM10, SO2, NO2): {df_air.count():,} records")
    print(f"  🌤️  Weather: {df_weather.count():,} records")
    
    # Weather data - drop missing (ít missing)
    df_weather_clean = df_weather.na.drop()
    
    # Pivot pollutant data
    df_air_pivot = df_air.groupBy(
        "location_id", "location", "datetime", "lat", "lon"
    ).pivot("parameter").agg(F.first("value"))
    
    # Rename columns
    column_mapping = {
        "pm25": "PM2_5",
        "pm10": "PM10",
        "no2": "NO2",
        "so2": "SO2"
    }
    
    for old_name, new_name in column_mapping.items():
        if old_name in df_air_pivot.columns:
            df_air_pivot = df_air_pivot.withColumnRenamed(old_name, new_name)
    
    # Join với weather data (theo datetime)
    df_location = df_air_pivot.join(
        df_weather_clean,
        df_air_pivot.datetime == df_weather_clean.time,
        "inner"
    ).drop("time")
    
    print(f"  ✓ After join: {df_location.count():,} records")
    
    # Thêm vào list
    all_locations_data.append(df_location)

print(f"\n✅ Processed {len(all_locations_data)} locations successfully!")


🔄 Processing Location 233335...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 83,970 records
  🌤️  Weather: 25,560 records
  ✓ After join: 21,686 records

🔄 Processing Location 233336...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 83,818 records
  🌤️  Weather: 25,560 records
  ✓ After join: 21,262 records

🔄 Processing Location 7727...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 86,684 records
  🌤️  Weather: 25,560 records
  ✓ After join: 22,367 records

🔄 Processing Location 7728...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 84,638 records
  🌤️  Weather: 25,560 records
  ✓ After join: 21,683 records

🔄 Processing Location 7730...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 82,776 records
  🌤️  Weather: 25,560 records
  ✓ After join: 21,274 records

🔄 Processing Location 7732...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 83,386 records
  🌤️  Weather: 25,560 records
  ✓ After join: 21,306 records

🔄 Processing Location 7733...
  📊 Air quality (PM2.5, PM10, SO2, NO2): 83,047 records
  🌤️  Weather: 25,560

### 2.3 Gộp tất cả Locations

In [6]:
# Gộp tất cả locations lại
print(f"🔄 Combining {len(all_locations_data)} locations...")

df_combined = all_locations_data[0]
for df in all_locations_data[1:]:
    df_combined = df_combined.union(df)

# OPTIMIZE: Cache để tránh recompute nhiều lần
df_combined = df_combined.cache()

# OPTIMIZE: Trigger action 1 lần, tránh count() nhiều lần
print("⏳ Computing combined dataset (this may take a moment)...")
total_records = df_combined.count()
num_locations = df_combined.select('location_id').distinct().count()

print(f"✅ Combined dataset: {total_records:,} total records")
print(f"✅ Number of locations: {num_locations}")

# OPTIMIZE: Chỉ show sample, không orderBy toàn bộ dataset (rất chậm!)
print("\n📋 Sample records (unsorted):")
df_combined.show(10, truncate=False)

# OPTIONAL: Nếu cần sort, chỉ sort 1 partition nhỏ để xem
# df_combined.orderBy("location_id", "datetime").limit(50).show(truncate=False)

🔄 Combining 14 locations...
⏳ Computing combined dataset (this may take a moment)...
✅ Combined dataset: 300,416 total records
✅ Number of locations: 14

📋 Sample records (unsorted):
+-----------+------------+-------------------+------------------+---------+----+----+-----+---+--------------+--------------------+--------------+------------------+----------------+-------------+
|location_id|location    |datetime           |lat               |lon      |NO2 |PM10|PM2_5|SO2|temperature_2m|relative_humidity_2m|wind_speed_10m|wind_direction_10m|surface_pressure|precipitation|
+-----------+------------+-------------------+------------------+---------+----+----+-----+---+--------------+--------------------+--------------+------------------+----------------+-------------+
|233335     |North-245631|2023-10-01 05:00:00|22.496710000000004|114.12824|25.3|26.4|14.1 |1.6|24.6          |99.0                |2.8           |220.0             |1007.5          |0.0          |
|233335     |North-245631|202

## 3. Tổng quan Dataset

In [7]:
# Thống kê theo location
print("📊 Dataset Overview by Location:")
df_combined.groupBy("location_id", "location").count().orderBy("location_id").show(truncate=False)

# Time range của từng location
print("\n📅 Time Range by Location:")
df_combined.groupBy("location_id").agg(
    F.min("datetime").alias("start_date"),
    F.max("datetime").alias("end_date"),
    F.count("*").alias("records")
).orderBy("location_id").show(truncate=False)

📊 Dataset Overview by Location:
+-----------+--------------------+-----+
|location_id|location            |count|
+-----------+--------------------+-----+
|233335     |North-245631        |21686|
|233336     |Southern-245632     |21262|
|7727       |Tung Chung-7727     |22367|
|7728       |Mong Kok-7728       |21683|
|7730       |Central/Western-7730|21274|
|7732       |Causeway Bay-7732   |21306|
|7733       |Sha Tin-7733        |21235|
|7734       |Sham Shui Po-7734   |21254|
|7735       |Kwun Tong-7735      |21277|
|7736       |Kwai Chung-7736     |21281|
|7737       |Tai Po-7737         |21275|
|7739       |Yuen Long-7739      |21728|
|7740       |Tsuen Wan-7740      |21694|
|7742       |Tuen Mun-7742       |2375 |
|7742       |Tuen Mun-932161     |18719|
+-----------+--------------------+-----+


📅 Time Range by Location:
+-----------+-------------------+-------------------+-------+
|location_id|start_date         |end_date           |records|
+-----------+-------------------+----

In [8]:
# Kiểm tra missing values
print("⚠️  Missing Values Summary:")
for col_name in df_combined.columns:
    null_count = df_combined.filter(F.col(col_name).isNull()).count()
    total = df_combined.count()
    pct = (null_count / total) * 100
    if null_count > 0:  # Chỉ hiển thị cột có missing
        print(f"  {col_name:25s}: {null_count:8,} ({pct:6.2f}%)")

⚠️  Missing Values Summary:
  NO2                      :    7,620 (  2.54%)
  PM10                     :    3,391 (  1.13%)
  PM2_5                    :   11,161 (  3.72%)
  SO2                      :    7,432 (  2.47%)


In [9]:
# Statistics tổng quan
print("📈 Overall Statistics:")
df_combined.select(
    "PM2_5", "PM10", "NO2", "SO2",
    "temperature_2m", "relative_humidity_2m", "wind_speed_10m", "precipitation"
).describe().show()

📈 Overall Statistics:


+-------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|summary|             PM2_5|              PM10|               NO2|               SO2|    temperature_2m|relative_humidity_2m|    wind_speed_10m|     precipitation|
+-------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|  count|            289255|            297025|            292796|            292984|            300416|              300416|            300416|            300416|
|   mean|15.684557224594208|25.423372443397003|39.196086012103926| 3.720443778499844|23.179505752023825|   79.53433239241585|12.479650884107365|0.2698651203664245|
| stddev|10.895508720844566|19.754274618263263| 26.13457654994168|2.4376132869062395| 5.546174423257824|  15.460363021119056| 6.232915473285468|1.1481021462125374|
|    min|       

## 4. Làm sạch Dữ liệu

**Quy trình làm sạch:**
1. **Loại bỏ Outliers trước** - Để tránh giá trị cực đoan ảnh hưởng đến tính toán statistics
2. **Fill Missing Values sau** - Imputation dựa trên dữ liệu đã loại bỏ outliers

### 4.1. Loại bỏ Outliers

Loại bỏ các giá trị cực đoan trước khi imputation:

In [10]:
# Loại bỏ outliers theo WHO/EPA International Standards (cho dữ liệu Hong Kong)
# ⚠️  QUAN TRỌNG: PM2.5 là TARGET variable - PHẢI có giá trị thật!
#     → Records có PM2.5 = null sẽ BỊ LOẠI BỎ
#     → Chỉ các features khác (PM10, NO2, SO2) mới được phép null và impute sau

df_no_outliers = df_combined.filter(
    # 🎯 TARGET: PM2.5 theo WHO Emergency threshold (không cho phép null)
    (F.col("PM2_5").isNotNull()) & 
    (F.col("PM2_5") >= 0) & (F.col("PM2_5") < 250) &  # WHO Emergency: 250 μg/m³
    
    # 📊 FEATURES: WHO/EPA International Standards - Cho phép null, chỉ loại outliers
    ((F.col("PM10").isNull()) | ((F.col("PM10") >= 0) & (F.col("PM10") < 430))) &  # WHO Emergency: 430 μg/m³
    ((F.col("NO2").isNull()) | ((F.col("NO2") >= 0) & (F.col("NO2") < 400))) &     # WHO/EU: 400 μg/m³ (1-hour)
    ((F.col("SO2").isNull()) | ((F.col("SO2") >= 0) & (F.col("SO2") < 500))) &     # WHO/EU: 500 μg/m³ (10-min)
    
    # 🌤️ WEATHER: WMO standards cho Hong Kong
    (F.col("precipitation") >= 0) & (F.col("precipitation") < 100)  # WMO: 100mm/h extreme rain
)

records_before = df_combined.count()
records_after = df_no_outliers.count()
removed = records_before - records_after

print(f"📊 Outlier Removal:")
print(f"  Before: {records_before:,} records")
print(f"  After:  {records_after:,} records")
print(f"  Removed: {removed:,} records ({removed/records_before*100:.2f}%)")
print(f"\n  ⚠️  Note: Records with PM2.5 = null are REMOVED (target variable must have real values)")

# Kiểm tra missing values sau khi loại outliers
print("\n⚠️  Missing values after outlier removal:")
for col_name in ["PM2_5", "PM10", "NO2", "SO2"]:
    if col_name in df_no_outliers.columns:
        null_count = df_no_outliers.filter(F.col(col_name).isNull()).count()
        total = df_no_outliers.count()
        pct = (null_count / total) * 100
        if null_count > 0:
            print(f"  {col_name:10s}: {null_count:8,} ({pct:6.2f}%)")
        elif col_name == "PM2_5":
            print(f"  {col_name:10s}: {null_count:8,} ({pct:6.2f}%) ✅ (Target - must be 0%)")

📊 Outlier Removal:
  Before: 300,416 records
  After:  289,255 records
  Removed: 11,161 records (3.72%)

  ⚠️  Note: Records with PM2.5 = null are REMOVED (target variable must have real values)

⚠️  Missing values after outlier removal:
  PM2_5     :        0 (  0.00%) ✅ (Target - must be 0%)
  PM2_5     :        0 (  0.00%) ✅ (Target - must be 0%)
  PM10      :      296 (  0.10%)
  PM10      :      296 (  0.10%)
  NO2       :    7,369 (  2.55%)
  NO2       :    7,369 (  2.55%)
  SO2       :    7,186 (  2.48%)
  SO2       :    7,186 (  2.48%)


### 4.2. Xử lý Missing Values (Interpolation)

**Chiến lược Imputation cho Time Series:**
- **PM2.5**: Đã loại bỏ tất cả records có null (target variable)
- **PM10, NO2, SO2**: Sử dụng **Linear Interpolation** (tốt nhất cho time series)
  - Bước 1: **Linear Interpolation** - Nội suy tuyến tính dựa trên giá trị trước & sau
  - Bước 2: **Forward Fill** - Xử lý missing ở cuối chuỗi (không có giá trị sau)
  - Bước 3: **Backward Fill** - Xử lý missing ở đầu chuỗi (không có giá trị trước)
  - Bước 4: **Mean** - Backup cuối cùng (nếu còn missing)

In [11]:
# Chiến lược Imputation cho Time Series Data
# Sử dụng PySpark Window Functions - Nội suy tuyến tính dựa trên khoảng cách thời gian

# List các cột FEATURES cần impute (KHÔNG bao gồm PM2.5 - target variable)
pollutant_cols = ["PM10", "NO2", "SO2"]  # ⚠️ Không có PM2.5!

print(f"🔄 Time Series Imputation Strategy (PySpark Native):")
print(f"   1. True Linear Interpolation - y = y₁ + (y₂-y₁) × (t-t₁)/(t₂-t₁)")
print(f"   2. Forward Fill - If only prev value available")
print(f"   3. Backward Fill - If only next value available")
print(f"   4. Null - If no surrounding values (rare)")
print(f"\n   Columns to impute: {pollutant_cols}")
print(f"   PM2.5 NOT imputed (target variable - already removed nulls)")
print(f"   🔒 Safe: Window partitioned by location_id (no cross-location interpolation)\n")

# Cache để tăng performance
df_filled = df_no_outliers.cache()

# Kiểm tra missing TRƯỚC khi interpolate
print("⚠️  Missing values BEFORE interpolation:")
for col_name in pollutant_cols:
    if col_name in df_filled.columns:
        null_count = df_filled.filter(F.col(col_name).isNull()).count()
        total = df_filled.count()
        pct = (null_count / total) * 100
        if null_count > 0:
            print(f"  {col_name:10s}: {null_count:8,} ({pct:6.2f}%)")

🔄 Time Series Imputation Strategy (PySpark Native):
   1. True Linear Interpolation - y = y₁ + (y₂-y₁) × (t-t₁)/(t₂-t₁)
   2. Forward Fill - If only prev value available
   3. Backward Fill - If only next value available
   4. Null - If no surrounding values (rare)

   Columns to impute: ['PM10', 'NO2', 'SO2']
   PM2.5 NOT imputed (target variable - already removed nulls)
   🔒 Safe: Window partitioned by location_id (no cross-location interpolation)

⚠️  Missing values BEFORE interpolation:
  PM10      :      296 (  0.10%)
  PM10      :      296 (  0.10%)
  NO2       :    7,369 (  2.55%)
  NO2       :    7,369 (  2.55%)
  SO2       :    7,186 (  2.48%)
  SO2       :    7,186 (  2.48%)


In [12]:
# Áp dụng True Linear Interpolation với PySpark (không dùng Pandas)
# Nội suy tuyến tính dựa trên khoảng cách thời gian THỰC (epoch)
# Window function đảm bảo KHÔNG nội suy chéo giữa các locations

print("🔄 Applying true linear interpolation per location (PySpark native)...")

# Tạo cột epoch (timestamp dạng số) để tính toán khoảng cách thời gian
df_filled = df_filled.withColumn("epoch", F.col("datetime").cast("long"))

# Định nghĩa Window cho từng location
w_forward = (
    Window.partitionBy("location_id")
    .orderBy("epoch")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

w_backward = (
    Window.partitionBy("location_id")
    .orderBy("epoch")
    .rowsBetween(Window.currentRow, Window.unboundedFollowing)
)

# Xử lý từng pollutant column
for col_name in pollutant_cols:
    if col_name not in df_filled.columns:
        continue
    
    print(f"  ▶ Interpolating {col_name}...", end=" ", flush=True)
    
    # Bước 1: Tìm giá trị & timestamp TRƯỚC và SAU gần nhất (có giá trị non-null)
    df_filled = (
        df_filled
        .withColumn(f"{col_name}_prev_value", F.last(col_name, True).over(w_forward))
        .withColumn(f"{col_name}_next_value", F.first(col_name, True).over(w_backward))
        .withColumn(f"{col_name}_prev_time", F.last(F.when(F.col(col_name).isNotNull(), F.col("epoch")), True).over(w_forward))
        .withColumn(f"{col_name}_next_time", F.first(F.when(F.col(col_name).isNotNull(), F.col("epoch")), True).over(w_backward))
    )
    
    # Bước 2: Tính toán Linear Interpolation theo công thức:
    # y = y₁ + (y₂ - y₁) * (t - t₁) / (t₂ - t₁)
    interpolated_value = (
        F.col(f"{col_name}_prev_value") +
        (F.col(f"{col_name}_next_value") - F.col(f"{col_name}_prev_value")) *
        ((F.col("epoch") - F.col(f"{col_name}_prev_time")) /
         (F.col(f"{col_name}_next_time") - F.col(f"{col_name}_prev_time")))
    )
    
    # Bước 3: Logic chọn giá trị cuối cùng với fallback
    df_filled = df_filled.withColumn(
        col_name,
        F.when(F.col(col_name).isNotNull(), F.col(col_name))  # Giữ nguyên nếu có giá trị
         .when(
             # Linear interpolation nếu có cả prev & next và không chia 0
             (F.col(f"{col_name}_prev_value").isNotNull()) &
             (F.col(f"{col_name}_next_value").isNotNull()) &
             ((F.col(f"{col_name}_next_time") - F.col(f"{col_name}_prev_time")) != 0),
             interpolated_value
         )
         .when(F.col(f"{col_name}_prev_value").isNotNull(), F.col(f"{col_name}_prev_value"))  # Forward fill
         .when(F.col(f"{col_name}_next_value").isNotNull(), F.col(f"{col_name}_next_value"))  # Backward fill
         .otherwise(None)  # Vẫn null nếu không có data nào
    )
    
    # Bước 4: Xóa các cột phụ để giảm memory
    df_filled = df_filled.drop(
        f"{col_name}_prev_value", f"{col_name}_next_value",
        f"{col_name}_prev_time", f"{col_name}_next_time"
    )
    
    print("✓")

# Cache kết quả sau khi interpolation
df_filled = df_filled.cache()

# Trigger computation và đếm records
count = df_filled.count()
print(f"\n✅ Linear interpolation completed! Total records: {count:,}")
print(f"   ⚙️  Method: True linear interpolation based on time distance (epoch)")
print(f"   🔒 Safe: No cross-location interpolation (partitioned by location_id)")
print(f"   🚀 Optimized: Native PySpark (no Pandas conversion)")

🔄 Applying true linear interpolation per location (PySpark native)...
  ▶ Interpolating PM10... ✓
  ▶ Interpolating NO2... ✓
  ▶ Interpolating NO2... ✓
  ▶ Interpolating SO2... ✓
  ▶ Interpolating SO2... ✓
✓

✅ Linear interpolation completed! Total records: 289,255
   ⚙️  Method: True linear interpolation based on time distance (epoch)
   🔒 Safe: No cross-location interpolation (partitioned by location_id)
   🚀 Optimized: Native PySpark (no Pandas conversion)

✅ Linear interpolation completed! Total records: 289,255
   ⚙️  Method: True linear interpolation based on time distance (epoch)
   🔒 Safe: No cross-location interpolation (partitioned by location_id)
   🚀 Optimized: Native PySpark (no Pandas conversion)


In [13]:
# Verify: PM2.5 không có null, các features khác không có null
print("\n📋 Final Missing Values Check (After Interpolation):")

# Kiểm tra PM2.5 (target)
pm25_nulls = df_filled.filter(F.col("PM2_5").isNull()).count()
print(f"  PM2_5 (Target): {pm25_nulls:,} nulls ✅ (Must be 0)")

# Kiểm tra features
total_nulls = 0
for col_name in pollutant_cols:
    if col_name in df_filled.columns:
        null_count = df_filled.filter(F.col(col_name).isNull()).count()
        total_nulls += null_count
        if null_count > 0:
            print(f"  {col_name:10s}: {null_count:,} nulls ⚠️")
        else:
            print(f"  {col_name:10s}: {null_count:,} nulls ✅")

# Xử lý edge case: Drop records còn null (không có giá trị xung quanh để interpolate)
if total_nulls > 0:
    print(f"\n⚠️  Found {total_nulls} remaining nulls (edge cases with no surrounding data)")
    print(f"   → Dropping these records to ensure data quality...")
    
    records_before_drop = df_filled.count()
    
    # Drop records có bất kỳ feature nào còn null
    for col_name in pollutant_cols:
        df_filled = df_filled.filter(F.col(col_name).isNotNull())
    
    records_after_drop = df_filled.count()
    dropped = records_before_drop - records_after_drop
    
    print(f"   Before drop: {records_before_drop:,} records")
    print(f"   After drop:  {records_after_drop:,} records")
    print(f"   Dropped:     {dropped:,} records ({dropped/records_before_drop*100:.2f}%)")
    print(f"\n   ✅ All feature columns now have 0 nulls!")
else:
    print("\n  ✅ No missing values in any feature columns!")

# Xóa cột epoch (đã dùng xong)
df_filled = df_filled.drop("epoch")

# Verify lần cuối
print(f"\n📊 Final Verification:")
for col_name in ["PM2_5"] + pollutant_cols:
    if col_name in df_filled.columns:
        null_count = df_filled.filter(F.col(col_name).isNull()).count()
        print(f"  {col_name:10s}: {null_count:,} nulls ✅")

print(f"\n✅ Data cleaning completed with True Linear Interpolation!")
print(f"   Final dataset: {df_filled.count():,} records")
print(f"   ⚠️  All records have REAL PM2.5 values (target variable)")
print(f"   ✅ Features interpolated smoothly (time-based linear interpolation)")
print(f"   ✅ Edge cases (no surrounding data) removed")
print(f"   🚀 Performance: Native PySpark (no Pandas conversion)")

# Cập nhật df_combined với dữ liệu đã clean và sắp xếp
df_combined = df_filled.orderBy("location_id", "datetime")


📋 Final Missing Values Check (After Interpolation):
  PM2_5 (Target): 0 nulls ✅ (Must be 0)
  PM2_5 (Target): 0 nulls ✅ (Must be 0)
  PM10      : 0 nulls ✅
  PM10      : 0 nulls ✅
  NO2       : 0 nulls ✅
  NO2       : 0 nulls ✅
  SO2       : 0 nulls ✅

  ✅ No missing values in any feature columns!

📊 Final Verification:
  SO2       : 0 nulls ✅

  ✅ No missing values in any feature columns!

📊 Final Verification:
  PM2_5     : 0 nulls ✅
  PM2_5     : 0 nulls ✅
  PM10      : 0 nulls ✅
  PM10      : 0 nulls ✅
  NO2       : 0 nulls ✅
  NO2       : 0 nulls ✅
  SO2       : 0 nulls ✅

✅ Data cleaning completed with True Linear Interpolation!
  SO2       : 0 nulls ✅

✅ Data cleaning completed with True Linear Interpolation!
   Final dataset: 289,255 records
   ⚠️  All records have REAL PM2.5 values (target variable)
   ✅ Features interpolated smoothly (time-based linear interpolation)
   ✅ Edge cases (no surrounding data) removed
   🚀 Performance: Native PySpark (no Pandas conversion)
   Fina

## 5. Feature Engineering & Normalization

**Quy trình ĐÚNG để tránh Data Leakage:**
1. **Time Features** - Thêm cyclic encoding (sin/cos) và is_weekend (không cần normalize)
2. **Temporal Split** - Chia train/validation/test theo thời gian (70/15/15)
3. **Normalization** - Chuẩn hóa **CHỈ numerical GỐC** bằng Min-Max từ train set
4. **Lag Features** - Tạo lag TỪ CÁC CỘT ĐÃ SCALE (giữ đúng scale relationship)
5. **Model-Specific Datasets** - Chuẩn bị riêng cho Deep Learning và XGBoost
6. **Null Handling** - Xử lý nulls trong lag features cuối cùng

**⚠️ QUAN TRỌNG:** Lag features phải tạo SAU khi normalize để giữ đúng mối quan hệ scale!

In [14]:
# Bước 1: Thêm Time Features từ dữ liệu đã clean
print("🔄 Step 1: Adding Time Features (No normalization needed)...")

import math

df_features = df_combined \
    .withColumn("hour", F.hour("datetime")) \
    .withColumn("month", F.month("datetime")) \
    .withColumn("day_of_week", F.dayofweek("datetime"))

# Cyclic encoding cho hour (24h cycle)
df_features = df_features \
    .withColumn("hour_sin", F.sin(2 * math.pi * F.col("hour") / 24)) \
    .withColumn("hour_cos", F.cos(2 * math.pi * F.col("hour") / 24))

# Cyclic encoding cho month (12 month cycle)
df_features = df_features \
    .withColumn("month_sin", F.sin(2 * math.pi * F.col("month") / 12)) \
    .withColumn("month_cos", F.cos(2 * math.pi * F.col("month") / 12))

# Cyclic encoding cho day_of_week (7 day cycle)
df_features = df_features \
    .withColumn("day_of_week_sin", F.sin(2 * math.pi * F.col("day_of_week") / 7)) \
    .withColumn("day_of_week_cos", F.cos(2 * math.pi * F.col("day_of_week") / 7))

# Binary feature: is_weekend
df_features = df_features \
    .withColumn("is_weekend", F.when(F.col("day_of_week").isin([1, 7]), 1).otherwise(0))

# Xóa các cột trung gian
df_features = df_features.drop("hour", "month", "day_of_week")

print("✓ Time features added successfully!")
print(f"✓ Total records: {df_features.count():,}")
print(f"✓ Total columns: {len(df_features.columns)}")

print("\n📋 Time Features Created:")
print("  Cyclic (sin/cos): hour, month, day_of_week → Already in [-1, 1]")
print("  Binary: is_weekend → Already in [0, 1]")
print("  ✅ No normalization needed for time features!")

🔄 Step 1: Adding Time Features (No normalization needed)...
✓ Time features added successfully!
✓ Time features added successfully!
✓ Total records: 289,255
✓ Total columns: 22

📋 Time Features Created:
  Cyclic (sin/cos): hour, month, day_of_week → Already in [-1, 1]
  Binary: is_weekend → Already in [0, 1]
  ✅ No normalization needed for time features!
✓ Total records: 289,255
✓ Total columns: 22

📋 Time Features Created:
  Cyclic (sin/cos): hour, month, day_of_week → Already in [-1, 1]
  Binary: is_weekend → Already in [0, 1]
  ✅ No normalization needed for time features!


In [ ]:
# Bước 2: TEMPORAL SPLIT TRƯỚC KHI NORMALIZE (Tránh Data Leakage)
print("\n🔄 Step 2: Temporal Train/Val/Test Split BEFORE Normalization...")

# Tính toán ngày chia dựa trên percentile thời gian  
time_stats = df_features.select(
    F.min("datetime").alias("min_time"),
    F.max("datetime").alias("max_time")
).collect()[0]

min_time = time_stats["min_time"]
max_time = time_stats["max_time"]
total_days = (max_time - min_time).days

# 70% train, 15% validation, 15% test
train_days = int(total_days * 0.70)
val_days = int(total_days * 0.15)

train_end = min_time + pd.Timedelta(days=train_days)
val_end = train_end + pd.Timedelta(days=val_days)

print(f"📅 Temporal Split (Avoiding Data Leakage):")
print(f"  🟢 Train:      {min_time.strftime('%Y-%m-%d')} → {train_end.strftime('%Y-%m-%d')} ({(train_end - min_time).days} days)")
print(f"  🟡 Validation: {train_end.strftime('%Y-%m-%d')} → {val_end.strftime('%Y-%m-%d')} ({(val_end - train_end).days} days)")
print(f"  🔴 Test:       {val_end.strftime('%Y-%m-%d')} → {max_time.strftime('%Y-%m-%d')} ({(max_time - val_end).days} days)")

# Split data - Count BEFORE caching to trigger evaluation
df_train_raw = df_features.filter(F.col("datetime") < train_end)
df_val_raw = df_features.filter((F.col("datetime") >= train_end) & (F.col("datetime") < val_end))
df_test_raw = df_features.filter(F.col("datetime") >= val_end)

train_count = df_train_raw.count()
val_count = df_val_raw.count() 
test_count = df_test_raw.count()
total_count = train_count + val_count + test_count

print(f"\n📊 Split Results:")
print(f"  🟢 Train: {train_count:8,} ({train_count/total_count*100:.1f}%)")
print(f"  🟡 Val:   {val_count:8,} ({val_count/total_count*100:.1f}%)")
print(f"  🔴 Test:  {test_count:8,} ({test_count/total_count*100:.1f}%)")

# Now cache after counting (avoids double computation)
df_train_raw = df_train_raw.cache()
df_val_raw = df_val_raw.cache()
df_test_raw = df_test_raw.cache()

print(f"\n✅ Temporal split completed!")
print(f"   ⚠️  Next: Normalize using TRAIN SET statistics ONLY")


🔄 Step 2: Temporal Train/Val/Test Split BEFORE Normalization...
📅 Temporal Split (Avoiding Data Leakage):
  🟢 Train:      2022-11-01 → 2024-11-14 (744 days)
  🟡 Validation: 2024-11-14 → 2025-04-22 (159 days)
  🔴 Test:       2025-04-22 → 2025-09-30 (161 days)
📅 Temporal Split (Avoiding Data Leakage):
  🟢 Train:      2022-11-01 → 2024-11-14 (744 days)
  🟡 Validation: 2024-11-14 → 2025-04-22 (159 days)
  🔴 Test:       2025-04-22 → 2025-09-30 (161 days)

📊 Split Results:
  🟢 Train:  187,636 (64.9%)
  🟡 Val:     49,925 (17.3%)
  🔴 Test:    51,694 (17.9%)

✅ Temporal split completed!
   ⚠️  Next: Normalize using TRAIN SET statistics ONLY

📊 Split Results:
  🟢 Train:  187,636 (64.9%)
  🟡 Val:     49,925 (17.3%)
  🔴 Test:    51,694 (17.9%)

✅ Temporal split completed!
   ⚠️  Next: Normalize using TRAIN SET statistics ONLY


In [ ]:
# Bước 3: Normalize NUMERICAL GỐC (CHỈ gốc, KHÔNG có lag features)
print(f"\n🔄 Step 3: Normalize NUMERICAL BASE FEATURES using TRAIN SET ONLY...")

# ⚠️ QUAN TRỌNG: CHỈ normalize các cột GỐC, KHÔNG bao gồm lag features
# Lag features sẽ tạo SAU từ các cột đã scale
numerical_base_cols = [
    # Pollutants (current values only)
    "PM2_5", "PM10", "NO2", "SO2",
    # Weather features (current values only)
    "temperature_2m", "relative_humidity_2m", "wind_speed_10m", 
    "wind_direction_10m", "precipitation"
]

print(f"📊 Normalizing {len(numerical_base_cols)} BASE features (NO lag features yet)...")
print(f"   Features to normalize: {numerical_base_cols}")
print(f"   ⚠️  Computing min/max from TRAIN SET ONLY (preventing data leakage)")

# Tính min/max CHỈ TỪ TRAIN SET
scaler_params = {}

for col_name in numerical_base_cols:
    if col_name in df_train_raw.columns:
        # CHỈ DÙNG TRAIN SET ĐỂ TÍNH MIN/MAX  
        stats = df_train_raw.select(
            F.min(col_name).alias("min"),
            F.max(col_name).alias("max")
        ).collect()[0]
        
        min_val = stats["min"]
        max_val = stats["max"]
        
        # ⚠️ CRITICAL: Handle None values from null columns
        if min_val is None or max_val is None:
            print(f"  ⚠️  Skipping {col_name}: All values are null")
            continue
        
        # ⚠️ CRITICAL: Tránh chia 0 khi min = max
        if max_val == min_val:
            max_val = min_val + 1
        
        scaler_params[col_name] = {"min": min_val, "max": max_val}
        print(f"  ✓ {col_name:30s}: [{min_val:8.2f}, {max_val:8.2f}] → [0, 1]")

print(f"\n✅ Scaler parameters computed from TRAIN SET only!")

# Áp dụng normalization cho tất cả splits
def apply_scaling(df, scaler_params):
    """Apply Min-Max scaling using precomputed parameters"""
    df_scaled = df
    for col_name, params in scaler_params.items():
        if col_name in df.columns:
            min_val = params["min"]
            max_val = params["max"]
            df_scaled = df_scaled.withColumn(
                f"{col_name}_scaled",
                (F.col(col_name) - min_val) / (max_val - min_val)
            )
    return df_scaled

print(f"\n? Applying Min-Max scaling [0, 1] to all splits...")

# Apply scaling and trigger computation
df_train = apply_scaling(df_train_raw, scaler_params)
df_val = apply_scaling(df_val_raw, scaler_params)
df_test = apply_scaling(df_test_raw, scaler_params)

# Trigger computation and cache
_ = df_train.count()
_ = df_val.count()
_ = df_test.count()

df_train = df_train.cache()
df_val = df_val.cache()
df_test = df_test.cache()

# Unpersist raw versions to free memory
df_train_raw.unpersist()
df_val_raw.unpersist()
df_test_raw.unpersist()

print(f"✅ Base feature normalization completed!")
print(f"   📊 All splits normalized using train statistics only")
print(f"   ⚠️  Next: Create lag features FROM SCALED COLUMNS")

In [ ]:
# Bước 4: Lưu Scaler Parameters
print(f"\n💾 Step 4: Saving Scaler Parameters...")

import json
from pathlib import Path

scaler_json = {
    col: {"min": float(params["min"]), "max": float(params["max"])} 
    for col, params in scaler_params.items()
}

# ========================================
# ADAPTIVE PATH (Kaggle vs Colab vs Local)
# ========================================
if IN_KAGGLE:
    # 🏆 Kaggle: Write to /kaggle/working (auto-saved on commit)
    processed_dir = Path("/kaggle/working/processed")
    print(f"🏆 Kaggle mode: Saving to {processed_dir}")
    
elif IN_COLAB:
    # 🌐 Colab: Write to Google Drive
    processed_dir = Path("/content/drive/MyDrive/pm25-data/processed")
    print(f"🌐 Colab mode: Saving to Google Drive")
    
else:
    # 💻 Local: Write to project folder
    processed_dir = Path("../data/processed")
    print(f"💻 Local mode: Saving to {processed_dir}")

# Tạo thư mục với parents=True (tạo cả parent directories nếu chưa có)
processed_dir.mkdir(parents=True, exist_ok=True)

# Lưu ra file JSON
scaler_path = processed_dir / "scaler_params.json"
with open(scaler_path, 'w') as f:
    json.dump(scaler_json, f, indent=2)

print(f"✅ Scaler parameters saved to: {scaler_path}")
print(f"   - Computed from TRAIN SET only (no data leakage)")
print(f"   - Used for denormalizing predictions during inference")
print(f"   - Contains {len(scaler_params)} base features")

# Hiển thị ví dụ
print(f"\n📋 Example scaler params (from train set):")
example_cols = ["PM2_5", "temperature_2m", "wind_speed_10m"]
for col in example_cols:
    if col in scaler_params:
        params = scaler_params[col]
        print(f"  {col:20s}: min={params['min']:.2f}, max={params['max']:.2f}")


💾 Scaler parameters saved to: ..\data\processed\scaler_params.json
   (Computed from TRAIN SET only - no data leakage)
   (Dùng để denormalize predictions khi inference)

📋 Example scaler params (from train set):
  PM2_5               : min=0.00, max=152.40
  temperature_2m      : min=1.90, max=36.00
  wind_speed_10m      : min=0.00, max=82.30


In [ ]:
# Bước 5: Tạo Lag Features TỪ CÁC CỘT ĐÃ SCALE (CHỈ CHO XGBOOST)
print(f"\n🔄 Step 5: Creating Lag Features FROM SCALED COLUMNS (XGBoost only)...")

# ⚠️ QUAN TRỌNG: Lag features được tạo TỪ CÁC CỘT ĐÃ SCALE
# → Đảm bảo lag và gốc có CÙNG SCALE PARAMETERS
# → Giữ đúng mối quan hệ giữa giá trị hiện tại và quá khứ

LAG_STEPS = [1, 2, 3, 6, 12, 24]  # 1h, 2h, 3h, 6h, 12h, 24h trước

# Columns cần tạo lag (sử dụng bản SCALED)
lag_base_columns = ["PM2_5", "PM10", "NO2", "SO2", 
                    "temperature_2m", "relative_humidity_2m", 
                    "wind_speed_10m", "precipitation"]

print(f"\n📋 Creating lag features:")
print(f"   Deep Learning models: No lags needed (learn from sequences)")
print(f"   XGBoost: {len(LAG_STEPS)} lags × {len(lag_base_columns)} variables = {len(LAG_STEPS) * len(lag_base_columns)} features")
print(f"   ✅ Using SCALED columns as source (proper scale relationship)")

# Window cho từng location (sắp xếp theo thời gian)
w_lag = Window.partitionBy("location_id").orderBy("datetime")

# Tạo lag features cho từng split (train, val, test)
def create_lag_features(df, lag_base_columns, lag_steps):
    """Create lag features from SCALED columns"""
    df_with_lags = df
    
    for col_name in lag_base_columns:
        col_scaled = f"{col_name}_scaled"
        
        if col_scaled in df.columns:
            for lag in lag_steps:
                lag_col_name = f"{col_name}_lag{lag}_scaled"
                
                # ✅ Tạo lag TỪ CỘT ĐÃ SCALE
                df_with_lags = df_with_lags.withColumn(
                    lag_col_name,
                    F.lag(col_scaled, lag).over(w_lag)
                )
    
    return df_with_lags

# Apply to all splits
print(f"\n🔄 Creating lag features for all splits...")
df_train = create_lag_features(df_train, lag_base_columns, LAG_STEPS)
df_val = create_lag_features(df_val, lag_base_columns, LAG_STEPS)
df_test = create_lag_features(df_test, lag_base_columns, LAG_STEPS)

print(f"  ✓ Train: {len(LAG_STEPS) * len(lag_base_columns)} lag features created")
print(f"  ✓ Val:   {len(LAG_STEPS) * len(lag_base_columns)} lag features created")
print(f"  ✓ Test:  {len(LAG_STEPS) * len(lag_base_columns)} lag features created")

# Trigger computation and cache
_ = df_train.count()
_ = df_val.count()
_ = df_test.count()

df_train = df_train.cache()
df_val = df_val.cache()
df_test = df_test.cache()

print(f"\n✅ Lag features created successfully!")
print(f"   ✅ All lags created FROM SCALED columns")
print(f"   ✅ Lag and base features have SAME scale parameters")
print(f"   ✅ Proper temporal relationship preserved")

# ========================================
# XỬ LÝ NULL VALUES TRONG LAG FEATURES
# ========================================
print(f"\n🔄 Handling null values in lag features...")

# Tạo list tất cả lag feature names
lag_feature_names = [f"{col}_lag{lag}_scaled" for col in lag_base_columns for lag in LAG_STEPS]

# Đếm nulls TRƯỚC khi xử lý
print(f"\n📊 Null counts BEFORE handling:")
sample_lag_features = lag_feature_names[:3]
for lag_col in sample_lag_features:
    if lag_col in df_train.columns:
        null_count = df_train.filter(F.col(lag_col).isNull()).count()
        total_count = df_train.count()
        print(f"  {lag_col:35s}: {null_count:8,} nulls ({null_count/total_count*100:.2f}%)")

print(f"\n⚠️  Reason: First {max(LAG_STEPS)} hours of each location have no previous data")
print(f"   Strategy: DROP records with ANY null lag feature")

# Track counts before drop
train_before = df_train.count()
val_before = df_val.count()
test_before = df_test.count()

# Function to drop nulls
def drop_lag_nulls(df, lag_features):
    """Drop records with any null lag feature"""
    df_clean = df
    for col in lag_features:
        if col in df.columns:
            df_clean = df_clean.filter(F.col(col).isNotNull())
    return df_clean

# Apply to all splits
print(f"\n🗑️  Dropping records with null lag features...")
df_train_clean = drop_lag_nulls(df_train, lag_feature_names)
df_val_clean = drop_lag_nulls(df_val, lag_feature_names)
df_test_clean = drop_lag_nulls(df_test, lag_feature_names)

# Count after
train_after = df_train_clean.count()
val_after = df_val_clean.count()
test_after = df_test_clean.count()

# Cache cleaned datasets
df_train_clean = df_train_clean.cache()
df_val_clean = df_val_clean.cache()
df_test_clean = df_test_clean.cache()

# Unpersist old ones
df_train.unpersist()
df_val.unpersist()
df_test.unpersist()

# Reassign
df_train = df_train_clean
df_val = df_val_clean
df_test = df_test_clean

print(f"\n📊 Records dropped (null lag features):")
print(f"  🟢 Train: {train_before:,} → {train_after:,} (dropped {train_before - train_after:,}, {(train_before - train_after)/train_before*100:.2f}%)")
print(f"  🟡 Val:   {val_before:,} → {val_after:,} (dropped {val_before - val_after:,}, {(val_before - val_after)/val_before*100:.2f}%)")
print(f"  🔴 Test:  {test_before:,} → {test_after:,} (dropped {test_before - test_after:,}, {(test_before - test_after)/test_before*100:.2f}%)")

# Verify no nulls
print(f"\n✅ Verification - checking for remaining nulls...")
sample_check = lag_feature_names[:3]
total_nulls_after = 0
for lag_col in sample_check:
    if lag_col in df_train.columns:
        null_count = df_train.filter(F.col(lag_col).isNull()).count()
        total_nulls_after += null_count
        status = "✅" if null_count == 0 else "❌"
        print(f"  {lag_col:35s}: {null_count:8,} nulls {status}")

if total_nulls_after == 0:
    print(f"\n✅ All lag features are clean!")
else:
    print(f"\n⚠️  Still {total_nulls_after} nulls found!")

print(f"\n✅ Lag features + Null handling completed!")
print(f"   - Created {len(lag_feature_names)} lag features FROM SCALED columns")
print(f"   - Lost only first {max(LAG_STEPS)} hours per location")
print(f"   - All lag features now have valid values")
print(f"   - Data quality ensured for XGBoost training")

In [ ]:
# Bước 6: Chuẩn bị Features cho từng Model
print("\n🔄 Step 6: Preparing Model-Specific Features...")

# ========================================
# FEATURES CHO DEEP LEARNING MODELS (CNN1D-BLSTM, LSTM)
# ========================================
# Không cần lag features vì models tự học temporal patterns từ sequences

dl_input_features = []

# 1. Pollutants scaled (trừ PM2_5 - đây là target)
dl_input_features.extend(["PM10_scaled", "NO2_scaled", "SO2_scaled"])

# 2. Weather features scaled (core features)
dl_input_features.extend([
    "temperature_2m_scaled", "relative_humidity_2m_scaled", 
    "wind_speed_10m_scaled", "wind_direction_10m_scaled", "precipitation_scaled"
])

# 3. Time features (cyclic encoding - đã ở dạng sin/cos trong [-1, 1])
dl_input_features.extend([
    "hour_sin", "hour_cos", 
    "month_sin", "month_cos",
    "day_of_week_sin", "day_of_week_cos"
])

# 4. Time features (binary)
dl_input_features.extend(["is_weekend"])

print(f"🧠 DEEP LEARNING Features: {len(dl_input_features)} features")
print(f"   - Current pollutants (scaled): 3")
print(f"   - Weather (scaled): 5") 
print(f"   - Time (cyclic): 6")
print(f"   - Time (binary): 1")
print(f"   - NO LAG FEATURES (models learn from sequences)")

# ========================================  
# FEATURES CHO XGBOOST
# ========================================
# Cần lag features vì không có khả năng xử lý sequences

xgb_input_features = dl_input_features.copy()  # Start with DL features

# Thêm lag features CHỈ CHO XGBOOST (đã được tạo từ scaled columns)
for col_name in lag_base_columns:
    for lag in LAG_STEPS:
        lag_col_name = f"{col_name}_lag{lag}_scaled"
        xgb_input_features.append(lag_col_name)

print(f"\n📊 XGBOOST Features: {len(xgb_input_features)} features")
print(f"   - Deep Learning base features: {len(dl_input_features)}")
print(f"   - Lag features (from scaled columns): {len(lag_base_columns) * len(LAG_STEPS)}")
print(f"   - Total: {len(xgb_input_features)} features")

# Target variable (đã scaled)
target_feature = "PM2_5_scaled"

print(f"\n✅ Model-specific features prepared:")
print(f"  🧠 CNN1D-BLSTM-Attention: {len(dl_input_features)} features")
print(f"  🧠 LSTM: {len(dl_input_features)} features")  
print(f"  📊 XGBoost: {len(xgb_input_features)} features")
print(f"  🎯 Target: {target_feature}")

# ⚠️ CRITICAL: Verify ALL columns exist
missing_dl = [col for col in dl_input_features if col not in df_train.columns]
missing_xgb = [col for col in xgb_input_features if col not in df_train.columns]
missing_target = target_feature not in df_train.columns

if missing_dl or missing_xgb or missing_target:
    print(f"\n❌ MISSING COLUMNS DETECTED:")
    if missing_dl: 
        print(f"  DL models: {missing_dl}")
    if missing_xgb: 
        print(f"  XGBoost: {missing_xgb[:5]}...")  # Show first 5
    if missing_target:
        print(f"  Target: {target_feature}")
    
    print(f"\n⚠️  Available scaled columns:")
    scaled_cols = [c for c in df_train.columns if c.endswith('_scaled')]
    print(f"  {scaled_cols[:10]}...")
    
    raise ValueError("Missing required feature columns! Check normalization step.")
else:
    print(f"\n✅ All feature columns exist in datasets!")


🔄 Step 5: Preparing Model-Specific Features...
🧠 DEEP LEARNING Features: 15 features
   - Current pollutants: 3
   - Weather: 5
   - Time (cyclic): 6
   - Time (linear): 1
   - NO LAG FEATURES (models learn from sequences)

📊 XGBOOST Features: 63 features
   - Deep Learning base features: 15
   - Lag features: 48
   - Total: 63 features

✅ Model-specific features prepared:
  🧠 CNN1D-BLSTM-Attention: 15 features
  🧠 LSTM: 15 features
  📊 XGBoost: 63 features
  🎯 Target: PM2_5_scaled

✅ All feature columns exist in datasets!


In [ ]:
# Bước 7: Prepare Final Model Datasets
print("\n🔄 Step 7: Preparing Final Model-Specific Datasets...")

# ========================================
# DEEP LEARNING DATASETS (CNN1D-BLSTM & LSTM)
# ========================================
# Không cần lag features, chỉ cần base features + time features

print(f"\n🧠 Deep Learning datasets (no lag features):")

# Select only DL features + target
dl_train = df_train.select("location_id", "datetime", target_feature, *dl_input_features)
dl_val = df_val.select("location_id", "datetime", target_feature, *dl_input_features)
dl_test = df_test.select("location_id", "datetime", target_feature, *dl_input_features)

# Cache
dl_train = dl_train.cache()
dl_val = dl_val.cache()
dl_test = dl_test.cache()

dl_train_count = dl_train.count()
dl_val_count = dl_val.count()
dl_test_count = dl_test.count()

print(f"  ✓ Train: {dl_train_count:,} records, {len(dl_input_features)} features")
print(f"  ✓ Val:   {dl_val_count:,} records, {len(dl_input_features)} features")
print(f"  ✓ Test:  {dl_test_count:,} records, {len(dl_input_features)} features")

# ========================================
# XGBOOST DATASETS
# ========================================
# Cần cả base features + lag features

print(f"\n📊 XGBoost datasets (with lag features):")

# Select XGB features + target
xgb_train = df_train.select("location_id", "datetime", target_feature, *xgb_input_features)
xgb_val = df_val.select("location_id", "datetime", target_feature, *xgb_input_features)
xgb_test = df_test.select("location_id", "datetime", target_feature, *xgb_input_features)

# Cache
xgb_train = xgb_train.cache()
xgb_val = xgb_val.cache()
xgb_test = xgb_test.cache()

xgb_train_count = xgb_train.count()
xgb_val_count = xgb_val.count()
xgb_test_count = xgb_test.count()

print(f"  ✓ Train: {xgb_train_count:,} records, {len(xgb_input_features)} features")
print(f"  ✓ Val:   {xgb_val_count:,} records, {len(xgb_input_features)} features")
print(f"  ✓ Test:  {xgb_test_count:,} records, {len(xgb_input_features)} features")

print(f"\n✅ Final datasets prepared!")
print(f"   🧠 Deep Learning: {len(dl_input_features)} features (no lags)")
print(f"   📊 XGBoost: {len(xgb_input_features)} features (with {len(lag_base_columns) * len(LAG_STEPS)} lags)")
print(f"   🎯 Target: {target_feature}")
print(f"   ✅ All datasets cleaned and ready for training!")


📊 FEATURE ENGINEERING SUMMARY

1️⃣ Dataset Statistics:
   - Total records: 289,050
   - Total records: 289,050
   - Total locations: 14
   - Time range: 2022-11-01 → 2025-09-30

2️⃣ Feature Categories:
   📌 Deep Learning Features: 15 features
      - Current Pollutants (scaled): PM10, NO2, SO2
      - Current Weather (scaled): temperature_2m, relative_humidity_2m, wind_speed_10m, wind_direction_10m, precipitation
      - Time Features (cyclic): hour_sin/cos, month_sin/cos, day_of_week_sin/cos
      - Time Features (linear): is_weekend
   📌 XGBoost Features: 63 features
      - Deep Learning features: 15
      - Lag Features: 48 features
      - Variables: PM2_5, PM10, NO2, SO2, temperature_2m, relative_humidity_2m, wind_speed_10m, precipitation
      - Lag steps: [1, 2, 3, 6, 12, 24]

3️⃣ Target Variable:
   🎯 PM2_5_scaled (normalized PM2.5)

4️⃣ Normalization:
   ✅ All numerical features scaled to [0, 1] using train set only
   ✅ Scaler params saved to: scaler_params.json
   ✅ No dat

In [ ]:
# Bước 8: Feature Engineering Summary + Metadata Saving
print("\n" + "="*80)
print("📊 FEATURE ENGINEERING PIPELINE SUMMARY")
print("="*80)

print(f"\n✅ PIPELINE EXECUTION ORDER (Correct - No Data Leakage):")
print(f"   1️⃣ Time Features → Added cyclic (sin/cos) + is_weekend")
print(f"   2️⃣ Temporal Split → 70% train / 15% val / 15% test")
print(f"   3️⃣ Normalization → Min-Max [0,1] using TRAIN statistics ONLY")
print(f"   4️⃣ Lag Features + Null Handling → Created FROM SCALED columns, dropped nulls")
print(f"   5️⃣ Scaler Params → Saved for inference")
print(f"   6️⃣ Model Features → Prepared for Deep Learning & XGBoost")
print(f"   7️⃣ Final Datasets → Ready for training")

print(f"\n📊 DATASET STATISTICS:")
print(f"   Total records: {dl_train_count + dl_val_count + dl_test_count:,}")
print(f"   Total locations: {df_train.select('location_id').distinct().count()}")
print(f"   Time range: {min_time.strftime('%Y-%m-%d')} → {max_time.strftime('%Y-%m-%d')}")

print(f"\n📋 FEATURE BREAKDOWN:")
print(f"   🧠 Deep Learning (CNN1D-BLSTM & LSTM): {len(dl_input_features)} features")
print(f"      ├─ Pollutants (scaled): 3 (PM10, NO2, SO2)")
print(f"      ├─ Weather (scaled): 5 (temp, humidity, wind, precipitation)")
print(f"      ├─ Time (cyclic): 6 (hour, month, day_of_week → sin/cos)")
print(f"      └─ Time (binary): 1 (is_weekend)")
print(f"   ")
print(f"   📊 XGBoost: {len(xgb_input_features)} features")
print(f"      ├─ Deep Learning features: {len(dl_input_features)}")
print(f"      └─ Lag features: {len(lag_base_columns) * len(LAG_STEPS)} ({len(lag_base_columns)} vars × {len(LAG_STEPS)} lags)")

print(f"\n🎯 TARGET VARIABLE:")
print(f"   {target_feature} (normalized PM2.5 in [0, 1])")

print(f"\n✅ DATA QUALITY CHECKS:")
print(f"   ✓ No missing values in target")
print(f"   ✓ No missing values in features")
print(f"   ✓ No outliers (removed by WHO/EPA standards)")
print(f"   ✓ Proper temporal ordering")
print(f"   ✓ No data leakage (train/val/test temporally separated)")
print(f"   ✓ Correct scale relationship (lag from scaled columns)")
print(f"   ✓ No nulls in lag features (first {max(LAG_STEPS)}h dropped)")

print(f"\n💾 SAVED ARTIFACTS:")
print(f"   📁 scaler_params.json → Min-Max parameters (train set only)")
print(f"   📁 feature_metadata.json → Feature lists & configuration")

print(f"\n🚀 READY FOR NEXT PHASE:")
print(f"   Variables in memory:")
print(f"   - Deep Learning: dl_train, dl_val, dl_test")
print(f"   - XGBoost: xgb_train, xgb_val, xgb_test")
print(f"   Next step: Sequence creation for Deep Learning models")

print("="*80)

# ========================================
# SAVE FEATURE METADATA
# ========================================
# Lưu metadata về feature engineering để tham khảo trong tương lai

import json
from pathlib import Path

# Metadata cho feature engineering
dataset_metadata = {
    "project": "PM2.5 Prediction",
    "preprocessing_version": "2.0_refactored",
    "pipeline_order": [
        "Time Features (cyclic encoding)",
        "Temporal Split (70/15/15)",
        "Normalization (train stats only)",
        "Lag Features (from scaled columns)",
        "Null Handling (drop first 24h per location)"
    ],
    "deep_learning_features": dl_input_features,
    "xgboost_features": xgb_input_features,
    "target_feature": target_feature,
    "lag_config": {
        "lag_steps": LAG_STEPS,
        "lag_base_columns": lag_base_columns,
        "total_lag_features": len(lag_base_columns) * len(LAG_STEPS)
    },
    "temporal_split": {
        "train_end": train_end.isoformat(),
        "val_end": val_end.isoformat(),
        "min_time": min_time.isoformat(),
        "max_time": max_time.isoformat()
    },
    "dataset_counts": {
        "dl_train": dl_train_count,
        "dl_val": dl_val_count,
        "dl_test": dl_test_count,
        "xgb_train": xgb_train_count,
        "xgb_val": xgb_val_count,
        "xgb_test": xgb_test_count
    },
    "total_features": {
        "deep_learning": len(dl_input_features),
        "xgboost": len(xgb_input_features)
    }
}

# ========================================
# ADAPTIVE PATH (Kaggle vs Colab vs Local)
# ========================================
if IN_KAGGLE:
    # 🏆 Kaggle: Write to /kaggle/working (auto-saved on commit)
    processed_dir = Path("/kaggle/working/processed")
    print(f"\n🏆 Kaggle mode: Saving metadata to {processed_dir}")
    
elif IN_COLAB:
    # 🌐 Colab: Write to Google Drive
    processed_dir = Path("/content/drive/MyDrive/pm25-data/processed")
    print(f"\n🌐 Colab mode: Saving metadata to Google Drive")
    
else:
    # 💻 Local: Write to project folder
    processed_dir = Path("../data/processed")
    print(f"\n💻 Local mode: Saving metadata to {processed_dir}")

# Tạo thư mục với parents=True (tạo cả parent directories nếu chưa có)
processed_dir.mkdir(parents=True, exist_ok=True)

# Lưu metadata
metadata_path = processed_dir / "feature_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(dataset_metadata, f, indent=2)

print(f"\n💾 Feature metadata saved to: {metadata_path}")
print(f"   ✅ Pipeline version: 2.0 (refactored - no data leakage)")
print(f"   ✅ Contains: feature lists, lag config, split info, dataset counts")

In [ ]:
# Bước 9: Create Sequence Data for Deep Learning Models
print("\n🔄 Step 9: Creating Sequence Data for Deep Learning Models...")

import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, DoubleType

# Sequence configuration (optimized for Colab)
CNN_SEQUENCE_LENGTH = 48  # Optimal for long-term patterns
LSTM_SEQUENCE_LENGTH = 24  # Optimal for medium-term patterns

print(f"⚙️  Sequence Configuration:")
print(f"   - CNN1D-BLSTM-Attention: {CNN_SEQUENCE_LENGTH} timesteps")
print(f"   - LSTM: {LSTM_SEQUENCE_LENGTH} timesteps")

def create_sequences_optimized(df, feature_cols, target_col, sequence_length):
    """
    Optimized sequence creation with checkpointing to avoid StackOverflow
    
    🎯 Key Strategy:
    - Batch processing to avoid deep logical plans
    - Checkpoint after each batch to reset plan depth
    - Use broadcast joins for efficiency
    - Single final filter for null handling
    
    🔑 Null Handling (2-Layer Protection):
    Layer 1: Drop first N records/location (incomplete history)
    Layer 2: Filter ANY null in sequences (data gaps)
    Result: 100% clean sequences with ZERO nulls
    """
    print(f"    Creating {sequence_length}-step sequences...")
    
    window_spec = Window.partitionBy("location_id").orderBy("datetime")
    
    # ========================================
    # LAYER 1: Drop first N records (incomplete history)
    # ========================================
    df_base = df.select("location_id", "datetime", target_col, *feature_cols) \
                .repartition(4, "location_id") \
                .withColumn("row_num", F.row_number().over(window_spec)) \
                .filter(F.col("row_num") > sequence_length) \
                .drop("row_num") \
                .cache()
    
    records_after_layer1 = df_base.count()  # Materialize
    print(f"      🛡️  Layer 1: Dropped first {sequence_length} records/location")
    print(f"         Records: {records_after_layer1:,}")
    
    # ========================================
    # BATCH PROCESSING (避免 StackOverflow)
    # ========================================
    # Chia features thành batches nhỏ để tránh logical plan quá sâu
    BATCH_SIZE = 4  # Mỗi batch xử lý 4 features (4 × 48 lags = 192 ops - safe)
    feature_batches = [feature_cols[i:i+BATCH_SIZE] for i in range(0, len(feature_cols), BATCH_SIZE)]
    
    print(f"        📦 Processing {len(feature_batches)} batches ({len(feature_cols)} features)...")
    
    base_cols = ["location_id", "datetime"]
    result_df = df_base.select(*base_cols)
    
    for batch_idx, batch_features in enumerate(feature_batches, 1):
        print(f"           Batch {batch_idx}/{len(feature_batches)}: {len(batch_features)} features")
        
        # Tạo batch DataFrame
        batch_df = df_base.select(*base_cols, *batch_features)
        
        # Tạo sequences cho batch này
        for col_name in batch_features:
            # Tạo array of lags [t-1, t-2, ..., t-N]
            lag_exprs = [F.lag(col_name, step).over(window_spec) for step in range(1, sequence_length + 1)]
            batch_df = batch_df.withColumn(f"{col_name}_sequence", F.array(*lag_exprs))
        
        # Select chỉ sequence columns
        sequence_cols = [f"{col}_sequence" for col in batch_features]
        batch_df = batch_df.select(*base_cols, *sequence_cols).cache()
        batch_df.count()  # Materialize để reset logical plan
        
        # Join vào result
        result_df = result_df.join(batch_df, base_cols, "inner")
        
        # Unpersist batch (giải phóng memory)
        batch_df.unpersist()
    
    # ========================================
    # LAYER 2: Filter nulls in sequences
    # ========================================
    print(f"        🔍 Filtering null sequences...")
    
    all_sequence_cols = [f"{col}_sequence" for col in feature_cols]
    
    # Build null filter: ALL sequences must be NOT NULL
    from functools import reduce
    null_filter = reduce(
        lambda acc, col: acc & F.col(col).isNotNull(),
        all_sequence_cols,
        F.lit(True)
    )
    
    # Also check: NO null VALUES inside arrays (extra safety)
    # Trick: size(array) should equal sequence_length (nulls make size smaller)
    for seq_col in all_sequence_cols:
        null_filter = null_filter & (F.size(seq_col) == sequence_length)
    
    result_df = result_df.filter(null_filter)
    records_after_layer2 = result_df.count()
    dropped = records_after_layer1 - records_after_layer2
    
    if dropped > 0:
        print(f"      🛡️  Layer 2: Dropped {dropped:,} records with nulls")
        print(f"         Records: {records_after_layer1:,} → {records_after_layer2:,}")
    else:
        print(f"      ✅ Layer 2: No data gaps detected")
    
    # ========================================
    # FINAL: Add target and clean up
    # ========================================
    result_df = result_df.join(
        df_base.select("location_id", "datetime", target_col),
        ["location_id", "datetime"],
        "inner"
    ).filter(F.col(target_col).isNotNull()) \
     .withColumnRenamed(target_col, "target_value") \
     .cache()
    
    final_count = result_df.count()
    retention_rate = (final_count / records_after_layer1) * 100
    
    print(f"      ✅ Final: {final_count:,} records ({retention_rate:.1f}% retained)")
    
    # Cleanup
    df_base.unpersist()
    
    return result_df

print("\n📊 Creating sequences for each model...")

# Create CNN1D-BLSTM sequences
print(f"\n🧠 CNN1D-BLSTM-Attention ({CNN_SEQUENCE_LENGTH} timesteps):")
try:
    cnn_train_clean = create_sequences_optimized(dl_train, dl_input_features, target_feature, CNN_SEQUENCE_LENGTH)
    cnn_val_clean = create_sequences_optimized(dl_val, dl_input_features, target_feature, CNN_SEQUENCE_LENGTH)
    cnn_test_clean = create_sequences_optimized(dl_test, dl_input_features, target_feature, CNN_SEQUENCE_LENGTH)
    print(f"    ✅ CNN sequences created successfully")
except Exception as e:
    print(f"    ❌ CNN sequence creation failed: {str(e)[:100]}...")
    cnn_train_clean = cnn_val_clean = cnn_test_clean = None

# Create LSTM sequences  
print(f"\n🔄 LSTM ({LSTM_SEQUENCE_LENGTH} timesteps):")
try:
    lstm_train_clean = create_sequences_optimized(dl_train, dl_input_features, target_feature, LSTM_SEQUENCE_LENGTH)
    lstm_val_clean = create_sequences_optimized(dl_val, dl_input_features, target_feature, LSTM_SEQUENCE_LENGTH)
    lstm_test_clean = create_sequences_optimized(dl_test, dl_input_features, target_feature, LSTM_SEQUENCE_LENGTH)
    print(f"    ✅ LSTM sequences created successfully")
except Exception as e:
    print(f"    ❌ LSTM sequence creation failed: {str(e)[:100]}...")
    lstm_train_clean = lstm_val_clean = lstm_test_clean = None

print(f"\n✅ Sequence data preparation completed!")
print(f"\n📋 Data Quality Guarantee:")
print(f"   ✓ Layer 1: No incomplete history (first {CNN_SEQUENCE_LENGTH}/{LSTM_SEQUENCE_LENGTH} records dropped)")
print(f"   ✓ Layer 2: No data gaps in middle (nulls filtered out)")
print(f"   ✓ Result: 100% clean sequences with ZERO nulls")
print(f"   ✓ Ready for high-quality model training!")


🔄 Step 7: Creating Sequence Data for Deep Learning Models...
☁️ CLOUD ENVIRONMENT: Using optimal sequences
📊 Creating sequences for each model...

🧠 CNN1D-BLSTM-Attention (48 timesteps):
    Creating 48-step sequences...
        Processing batch 1/5...
        Processing batch 2/5...
        Processing batch 2/5...
        Processing batch 3/5...
        Processing batch 3/5...
        Processing batch 4/5...
        Processing batch 4/5...
        Processing batch 5/5...
        Processing batch 5/5...
    ❌ CNN sequence creation failed: An error occurred while calling o16487.count.
: org.apache.spark.SparkException: Job aborted due to ...

🔄 LSTM (24 timesteps):
    Creating 24-step sequences...
    ❌ CNN sequence creation failed: An error occurred while calling o16487.count.
: org.apache.spark.SparkException: Job aborted due to ...

🔄 LSTM (24 timesteps):
    Creating 24-step sequences...
        Processing batch 1/5...
        Processing batch 1/5...
    ❌ LSTM sequence creation f

In [ ]:
# Bước 10: Export Final Datasets to Disk
print("\n📦 Step 10: Exporting Final Datasets to Disk...")

import json
from pathlib import Path

# ========================================
# ADAPTIVE OUTPUT PATH (Kaggle vs Colab vs Local)
# ========================================
if IN_KAGGLE:
    # 🏆 Kaggle: Write to /kaggle/working (auto-saved on commit)
    processed_dir = Path("/kaggle/working/processed")
    print(f"🏆 Kaggle mode: Saving to {processed_dir}")
    print(f"   ⚠️  Files will be auto-saved when you commit notebook")
    
elif IN_COLAB:
    # 🌐 Colab: Write to Google Drive
    processed_dir = Path("/content/drive/MyDrive/pm25-data/processed")
    print(f"🌐 Colab mode: Saving to Google Drive")
    
else:
    # 💻 Local: Write to project folder
    processed_dir = Path("../data/processed")
    print(f"💻 Local mode: Saving to {processed_dir}")

processed_dir.mkdir(parents=True, exist_ok=True)

# Check dataset availability
datasets_ready = {
    "cnn": cnn_train_clean is not None and cnn_val_clean is not None and cnn_test_clean is not None,
    "lstm": lstm_train_clean is not None and lstm_val_clean is not None and lstm_test_clean is not None,
    "xgb": xgb_train is not None and xgb_val is not None and xgb_test is not None
}

print(f"\n📊 Dataset Status:")
for model, ready in datasets_ready.items():
    model_name = {"cnn": "CNN1D-BLSTM", "lstm": "LSTM", "xgb": "XGBoost"}[model]
    status = "✅ Ready" if ready else "❌ Not Ready"
    print(f"  {model_name}: {status}")

# ========================================
# EXPORT DATASETS TO PARQUET
# ========================================
print(f"\n💾 Exporting datasets to Parquet format...")

export_summary = {
    "cnn": {"train": 0, "val": 0, "test": 0},
    "lstm": {"train": 0, "val": 0, "test": 0},
    "xgb": {"train": 0, "val": 0, "test": 0}
}

# Export CNN1D-BLSTM datasets
if datasets_ready["cnn"]:
    print(f"\n  🧠 Exporting CNN1D-BLSTM datasets...")
    cnn_dir = processed_dir / "cnn_sequences"
    cnn_dir.mkdir(exist_ok=True)
    
    cnn_train_clean.write.mode("overwrite").parquet(str(cnn_dir / "train"))
    cnn_val_clean.write.mode("overwrite").parquet(str(cnn_dir / "val"))
    cnn_test_clean.write.mode("overwrite").parquet(str(cnn_dir / "test"))
    
    export_summary["cnn"]["train"] = cnn_train_clean.count()
    export_summary["cnn"]["val"] = cnn_val_clean.count()
    export_summary["cnn"]["test"] = cnn_test_clean.count()
    
    print(f"     ✅ Saved to: {cnn_dir}/")
    print(f"        - train: {export_summary['cnn']['train']:,} records")
    print(f"        - val:   {export_summary['cnn']['val']:,} records")
    print(f"        - test:  {export_summary['cnn']['test']:,} records")

# Export LSTM datasets
if datasets_ready["lstm"]:
    print(f"\n  🔄 Exporting LSTM datasets...")
    lstm_dir = processed_dir / "lstm_sequences"
    lstm_dir.mkdir(exist_ok=True)
    
    lstm_train_clean.write.mode("overwrite").parquet(str(lstm_dir / "train"))
    lstm_val_clean.write.mode("overwrite").parquet(str(lstm_dir / "val"))
    lstm_test_clean.write.mode("overwrite").parquet(str(lstm_dir / "test"))
    
    export_summary["lstm"]["train"] = lstm_train_clean.count()
    export_summary["lstm"]["val"] = lstm_val_clean.count()
    export_summary["lstm"]["test"] = lstm_test_clean.count()
    
    print(f"     ✅ Saved to: {lstm_dir}/")
    print(f"        - train: {export_summary['lstm']['train']:,} records")
    print(f"        - val:   {export_summary['lstm']['val']:,} records")
    print(f"        - test:  {export_summary['lstm']['test']:,} records")

# Export XGBoost datasets
if datasets_ready["xgb"]:
    print(f"\n  📊 Exporting XGBoost datasets...")
    xgb_dir = processed_dir / "xgboost"
    xgb_dir.mkdir(exist_ok=True)
    
    xgb_train.write.mode("overwrite").parquet(str(xgb_dir / "train"))
    xgb_val.write.mode("overwrite").parquet(str(xgb_dir / "val"))
    xgb_test.write.mode("overwrite").parquet(str(xgb_dir / "test"))
    
    export_summary["xgb"]["train"] = xgb_train.count()
    export_summary["xgb"]["val"] = xgb_val.count()
    export_summary["xgb"]["test"] = xgb_test.count()
    
    print(f"     ✅ Saved to: {xgb_dir}/")
    print(f"        - train: {export_summary['xgb']['train']:,} records")
    print(f"        - val:   {export_summary['xgb']['val']:,} records")
    print(f"        - test:  {export_summary['xgb']['test']:,} records")

# ========================================
# SAVE METADATA
# ========================================
print(f"\n💾 Saving metadata...")

# Create comprehensive metadata
final_metadata = {
    "project": "PM2.5 Prediction",
    "preprocessing_completed": True,
    "export_timestamp": str(pd.Timestamp.now()),
    "environment": "kaggle" if IN_KAGGLE else ("colab" if IN_COLAB else "local"),
    "models": {
        "cnn1d_blstm": {
            "sequence_length": CNN_SEQUENCE_LENGTH,
            "features": len(dl_input_features),
            "ready": datasets_ready["cnn"],
            "export_path": str(processed_dir / "cnn_sequences"),
            "record_counts": export_summary["cnn"]
        },
        "lstm": {
            "sequence_length": LSTM_SEQUENCE_LENGTH, 
            "features": len(dl_input_features),
            "ready": datasets_ready["lstm"],
            "export_path": str(processed_dir / "lstm_sequences"),
            "record_counts": export_summary["lstm"]
        },
        "xgboost": {
            "features": len(xgb_input_features),
            "lag_steps": LAG_STEPS,
            "ready": datasets_ready["xgb"],
            "export_path": str(processed_dir / "xgboost"),
            "record_counts": export_summary["xgb"]
        }
    },
    "feature_details": {
        "deep_learning_features": dl_input_features,
        "xgboost_features": xgb_input_features,
        "target": target_feature
    },
    "data_format": "parquet",
    "null_handling": {
        "strategy": "2-layer protection",
        "layer1": f"Dropped first {CNN_SEQUENCE_LENGTH}/{LSTM_SEQUENCE_LENGTH} records per location",
        "layer2": "Filtered records with nulls in sequence history"
    }
}

# Save metadata
metadata_path = processed_dir / "datasets_ready.json"
with open(metadata_path, 'w') as f:
    json.dump(final_metadata, f, indent=2)

print(f"   ✅ Metadata saved to: {metadata_path}")

# Save scaler params
scaler_path = processed_dir / "scaler_params.json"
scaler_json = {
    col: {"min": float(params["min"]), "max": float(params["max"])} 
    for col, params in scaler_params.items()
}
with open(scaler_path, 'w') as f:
    json.dump(scaler_json, f, indent=2)
print(f"   ✅ Scaler params saved to: {scaler_path}")

# Save feature metadata
feature_metadata_path = processed_dir / "feature_metadata.json"
feature_metadata = {
    "deep_learning_features": dl_input_features,
    "xgboost_features": xgb_input_features,
    "target": target_feature,
    "lag_steps": LAG_STEPS,
    "lag_base_columns": lag_base_columns
}
with open(feature_metadata_path, 'w') as f:
    json.dump(feature_metadata, f, indent=2)
print(f"   ✅ Feature metadata saved to: {feature_metadata_path}")

# ========================================
# FINAL SUMMARY
# ========================================
print(f"\n{'='*80}")
print(f"✅ DATA PREPROCESSING & EXPORT COMPLETE!")
print(f"{'='*80}")

if IN_KAGGLE:
    print(f"\n🏆 KAGGLE OUTPUT:")
    print(f"   📂 Location: /kaggle/working/processed/")
    print(f"   📌 To save permanently:")
    print(f"      1. Click 'Save Version' (top right)")
    print(f"      2. Choose 'Save & Run All' (recommended)")
    print(f"      3. Wait for completion (~20-30 min)")
    print(f"      4. Output will appear in 'Output' tab")
    print(f"      5. Use as dataset: '+ Add Data' → Your Output")
    
elif IN_COLAB:
    print(f"\n🌐 COLAB OUTPUT:")
    print(f"   📂 Saved to Google Drive: {processed_dir}")
    print(f"   ✅ Files persist across sessions")
    
else:
    print(f"\n💻 LOCAL OUTPUT:")
    print(f"   📂 Location: {processed_dir.absolute()}")

print(f"\n📂 Exported Directory Structure:")
print(f"   {processed_dir}/")
print(f"   ├── cnn_sequences/")
print(f"   │   ├── train/  ({export_summary['cnn']['train']:,} records)")
print(f"   │   ├── val/    ({export_summary['cnn']['val']:,} records)")
print(f"   │   └── test/   ({export_summary['cnn']['test']:,} records)")
print(f"   ├── lstm_sequences/")
print(f"   │   ├── train/  ({export_summary['lstm']['train']:,} records)")
print(f"   │   ├── val/    ({export_summary['lstm']['val']:,} records)")
print(f"   │   └── test/   ({export_summary['lstm']['test']:,} records)")
print(f"   ├── xgboost/")
print(f"   │   ├── train/  ({export_summary['xgb']['train']:,} records)")
print(f"   │   ├── val/    ({export_summary['xgb']['val']:,} records)")
print(f"   │   └── test/   ({export_summary['xgb']['test']:,} records)")
print(f"   ├── scaler_params.json")
print(f"   ├── feature_metadata.json")
print(f"   └── datasets_ready.json")

print(f"\n📊 Total Dataset Sizes:")
total_cnn = sum(export_summary['cnn'].values())
total_lstm = sum(export_summary['lstm'].values())
total_xgb = sum(export_summary['xgb'].values())
print(f"   - CNN1D-BLSTM: {total_cnn:,} records ({CNN_SEQUENCE_LENGTH} timesteps, {len(dl_input_features)} features)")
print(f"   - LSTM:        {total_lstm:,} records ({LSTM_SEQUENCE_LENGTH} timesteps, {len(dl_input_features)} features)")
print(f"   - XGBoost:     {total_xgb:,} records ({len(xgb_input_features)} features)")

print(f"\n🚀 Ready for Model Training Phase!")
if IN_KAGGLE:
    print(f"   📌 Next: Create new notebook, add this output as dataset")
    print(f"   📌 Load: spark.read.parquet('/kaggle/input/<output-name>/processed/...')")
elif IN_COLAB:
    print(f"   📌 Load from Drive in next session")
else:
    print(f"   📌 Load: spark.read.parquet('{processed_dir}/...')")
print(f"{'='*80}")


📦 Step 10: Exporting Final Datasets to Disk...


NameError: name 'cnn_train_clean' is not defined

In [ ]:
# ? Kiểm tra kích thước files Parquet đã export
print("\n? Parquet File Size Analysis...")
print("="*80)

import os

# ========================================
# ADAPTIVE PATH
# ========================================
if IN_KAGGLE:
    processed_dir = Path("/kaggle/working/processed")
    print(f"🏆 Kaggle mode: Analyzing {processed_dir}")
elif IN_COLAB:
    processed_dir = Path("/content/drive/MyDrive/pm25-data/processed")
    print(f"🌐 Colab mode: Analyzing Google Drive")
else:
    processed_dir = Path("../data/processed")
    print(f"💻 Local mode: Analyzing {processed_dir}")

# ========================================
# Tính kích thước thư mục
# ========================================
def get_dir_size(path):
    """Tính tổng kích thước của thư mục (bao gồm tất cả subdirectories)"""
    total_size = 0
    try:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                if os.path.exists(filepath):
                    total_size += os.path.getsize(filepath)
    except Exception as e:
        print(f"   ⚠️  Error accessing {path}: {e}")
        return 0
    return total_size

def format_size(bytes_size):
    """Format bytes thành human-readable"""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if bytes_size < 1024.0:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024.0
    return f"{bytes_size:.2f} TB"

# ========================================
# Phân tích từng dataset
# ========================================
print("\n📦 Dataset Sizes:")
print("-" * 80)

datasets = {
    'CNN Sequences': 'cnn_sequences',
    'LSTM Sequences': 'lstm_sequences',
    'XGBoost Data': 'xgboost'
}

total_size = 0
size_breakdown = {}

for name, folder in datasets.items():
    dataset_path = processed_dir / folder
    if dataset_path.exists():
        # Tính size cho từng split
        splits = ['train', 'val', 'test']
        dataset_total = 0
        print(f"\n🧠 {name}:")
        
        for split in splits:
            split_path = dataset_path / split
            if split_path.exists():
                size = get_dir_size(split_path)
                dataset_total += size
                print(f"   - {split:5s}: {format_size(size):>12s}")
        
        print(f"   {'Total:':7s} {format_size(dataset_total):>12s}")
        size_breakdown[name] = dataset_total
        total_size += dataset_total
    else:
        print(f"\n⚠️  {name}: Folder not found ({dataset_path})")

# Metadata files
print(f"\n? Metadata Files:")
metadata_files = ['scaler_params.json', 'feature_metadata.json', 'datasets_ready.json']
metadata_total = 0
for file in metadata_files:
    file_path = processed_dir / file
    if file_path.exists():
        size = os.path.getsize(file_path)
        metadata_total += size
        print(f"   - {file:25s}: {format_size(size):>12s}")
total_size += metadata_total

# ========================================
# Tổng kết
# ========================================
print("\n" + "="*80)
print("📊 TOTAL SIZE SUMMARY:")
print("="*80)

for name, size in size_breakdown.items():
    percentage = (size / total_size * 100) if total_size > 0 else 0
    print(f"   {name:20s}: {format_size(size):>12s} ({percentage:5.1f}%)")

print(f"   {'Metadata':20s}: {format_size(metadata_total):>12s} ({(metadata_total/total_size*100):5.1f}%)")
print(f"\n   {'GRAND TOTAL':20s}: {format_size(total_size):>12s}")

In [ ]:
# 🐼 EXAMPLE: Load Preprocessed Data with Pandas
print("\n🐼 Loading Preprocessed Data with Pandas...")
print("="*80)

import pandas as pd
import numpy as np

# ========================================
# ADAPTIVE PATH
# ========================================
if IN_KAGGLE:
    data_dir = Path("/kaggle/input/<your-dataset-name>/processed")
    print(f"🏆 Kaggle mode: Loading from /kaggle/input/")
    print(f"   Replace <your-dataset-name> with actual dataset name")
elif IN_COLAB:
    data_dir = Path("/content/drive/MyDrive/pm25-data/processed")
    print(f"🌐 Colab mode: Loading from Google Drive")
else:
    data_dir = Path("../data/processed")
    print(f"💻 Local mode: Loading from {data_dir}")

# ========================================
# LOAD PARQUET FILES
# ========================================
print("\n📦 Loading datasets...")

try:
    # CNN sequences (48 timesteps)
    print("\n🧠 CNN1D-BLSTM-Attention:")
    cnn_train = pd.read_parquet(data_dir / 'cnn_sequences' / 'train')
    cnn_val = pd.read_parquet(data_dir / 'cnn_sequences' / 'val')
    cnn_test = pd.read_parquet(data_dir / 'cnn_sequences' / 'test')
    print(f"   ✅ Train: {cnn_train.shape} | Val: {cnn_val.shape} | Test: {cnn_test.shape}")
    
    # LSTM sequences (24 timesteps)
    print("\n🔄 LSTM:")
    lstm_train = pd.read_parquet(data_dir / 'lstm_sequences' / 'train')
    lstm_val = pd.read_parquet(data_dir / 'lstm_sequences' / 'val')
    lstm_test = pd.read_parquet(data_dir / 'lstm_sequences' / 'test')
    print(f"   ✅ Train: {lstm_train.shape} | Val: {lstm_val.shape} | Test: {lstm_test.shape}")
    
    # XGBoost data (flat features)
    print("\n📊 XGBoost:")
    xgb_train = pd.read_parquet(data_dir / 'xgboost' / 'train')
    xgb_val = pd.read_parquet(data_dir / 'xgboost' / 'val')
    xgb_test = pd.read_parquet(data_dir / 'xgboost' / 'test')
    print(f"   ✅ Train: {xgb_train.shape} | Val: {xgb_val.shape} | Test: {xgb_test.shape}")
    
    print(f"\n✅ All datasets loaded successfully!")
    
except FileNotFoundError as e:
    print(f"\n❌ Error: Dataset not found!")
    print(f"   {e}")
    print(f"\n💡 Make sure to:")
    if IN_KAGGLE:
        print(f"   1. Add this notebook's output as dataset")
        print(f"   2. Update <your-dataset-name> in path")
    else:
        print(f"   1. Run previous cells to generate data")
        print(f"   2. Check path: {data_dir}")

# ========================================
# LOAD METADATA
# ========================================
print("\n📋 Loading metadata...")

import json

try:
    # Scaler parameters (for denormalization)
    with open(data_dir / 'scaler_params.json', 'r') as f:
        scaler_params = json.load(f)
    print(f"   ✅ Scaler params: {len(scaler_params)} features")
    
    # Feature metadata
    with open(data_dir / 'feature_metadata.json', 'r') as f:
        feature_metadata = json.load(f)
    print(f"   ✅ Feature metadata: {feature_metadata['preprocessing_version']}")
    
except FileNotFoundError:
    print(f"   ⚠️  Metadata files not found (optional)")
    scaler_params = None
    feature_metadata = None

# ========================================
# DATA PREPARATION FOR DEEP LEARNING
# ========================================
print("\n" + "="*80)
print("🎯 Prepare Data for Training:")
print("="*80)

# Example: CNN data preparation
print("\n📦 CNN1D-BLSTM data preparation:")

# Get sequence columns
sequence_cols = [col for col in cnn_train.columns if col.endswith('_sequence')]
print(f"   - Sequence features: {len(sequence_cols)}")
print(f"   - Feature names: {sequence_cols[:3]}... (showing first 3)")

# Convert to numpy arrays for deep learning
print("\n   Converting to numpy arrays...")

# Extract sequences (each row has arrays)
X_cnn_train = np.array([
    np.stack([cnn_train[col].iloc[i] for col in sequence_cols], axis=0)
    for i in range(len(cnn_train))
])  # Shape: (samples, features, timesteps)

# Transpose to (samples, timesteps, features) for Keras/PyTorch
X_cnn_train = X_cnn_train.transpose(0, 2, 1)
y_cnn_train = cnn_train['target_value'].values

print(f"   ✅ X_train shape: {X_cnn_train.shape} (samples, timesteps, features)")
print(f"   ✅ y_train shape: {y_cnn_train.shape}")

# Same for validation and test
X_cnn_val = np.array([
    np.stack([cnn_val[col].iloc[i] for col in sequence_cols], axis=0)
    for i in range(len(cnn_val))
]).transpose(0, 2, 1)
y_cnn_val = cnn_val['target_value'].values

X_cnn_test = np.array([
    np.stack([cnn_test[col].iloc[i] for col in sequence_cols], axis=0)
    for i in range(len(cnn_test))
]).transpose(0, 2, 1)
y_cnn_test = cnn_test['target_value'].values

print(f"   ✅ Val:  X={X_cnn_val.shape}, y={y_cnn_val.shape}")
print(f"   ✅ Test: X={X_cnn_test.shape}, y={y_cnn_test.shape}")

# ========================================
# EXAMPLE: XGBoost data preparation
# ========================================
print("\n📦 XGBoost data preparation:")

# XGBoost data is already flat (no sequences)
X_xgb_train = xgb_train.drop(['location_id', 'datetime', 'target_value'], axis=1).values
y_xgb_train = xgb_train['target_value'].values

X_xgb_val = xgb_val.drop(['location_id', 'datetime', 'target_value'], axis=1).values
y_xgb_val = xgb_val['target_value'].values

X_xgb_test = xgb_test.drop(['location_id', 'datetime', 'target_value'], axis=1).values
y_xgb_test = xgb_test['target_value'].values

print(f"   ✅ X_train shape: {X_xgb_train.shape} (samples, features)")
print(f"   ✅ y_train shape: {y_xgb_train.shape}")
print(f"   ✅ Val:  X={X_xgb_val.shape}, y={y_xgb_val.shape}")
print(f"   ✅ Test: X={X_xgb_test.shape}, y={y_xgb_test.shape}")

# ========================================
# SUMMARY
# ========================================
print("\n" + "="*80)
print("✅ DATA READY FOR TRAINING!")
print("="*80)

print("\n📊 Available datasets:")
print("   🧠 CNN1D-BLSTM-Attention:")
print(f"      X_cnn_train: {X_cnn_train.shape}")
print(f"      X_cnn_val:   {X_cnn_val.shape}")
print(f"      X_cnn_test:  {X_cnn_test.shape}")

print("\n   🔄 LSTM: (Similar structure, use lstm_train/val/test)")

print("\n   📊 XGBoost:")
print(f"      X_xgb_train: {X_xgb_train.shape}")
print(f"      X_xgb_val:   {X_xgb_val.shape}")
print(f"      X_xgb_test:  {X_xgb_test.shape}")

print("\n🚀 Next steps:")
print("   1. Build model: model = tf.keras.Sequential([...])")
print("   2. Compile: model.compile(optimizer='adam', loss='mse')")
print("   3. Train: model.fit(X_cnn_train, y_cnn_train, epochs=50)")
print("   4. Evaluate: model.evaluate(X_cnn_test, y_cnn_test)")

print("="*80)

In [ ]:
cnn_train.head(10)